# Logging

**Sources:**

- [Fast api logging essentials](https://marketsplash.com/fastapi-logging/);

## Set up

In [3]:
from requests import get

The following cell runs a Docker container that utilizes a specified file to build a FastAPI application.

In [114]:
!docker run --rm -itd\
    --name test_container\
    -v ./logging_files/app.py:/app.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload app:app

cd1bd3bcd1752b613f2f64f8dea566bcb8e99cf25c1d271c6efff43c576d0a4f


Don't forget to stop the container.

In [115]:
!docker stop test_container

test_container


## View logs

Let's see what the default logs of the fast api look like. We will make an application to write something to the console and then extract it from the docker container.

In the following cell we've just defined a simple fastapi program that throws a `HTTPException` if called with a `False` argument.

In [1]:
%%writefile logging_files/app.py
from fastapi import FastAPI, HTTPException

app = FastAPI()

@app.get("/{is_ok}")
def handle(is_ok : bool):
    if is_ok:
        return "im fine"
    else:
        raise HTTPException(status_code=404, detail="Item not found")

Overwriting logging_files/app.py


Now let's send some requests to the application and view it's log last lines:

In [14]:
get("http://localhost:8000/True")
get("http://localhost:8000/False")
!docker logs --tail 2 test_container

INFO:     172.17.0.1:37404 - "GET /True HTTP/1.1" 200 OK
INFO:     172.17.0.1:37420 - "GET /False HTTP/1.1" 404 Not Found


## Loggers

The following fastapi program returns all available loggers in the fast api.

In [116]:
%%writefile logging_files/app.py
import logging
from fastapi import FastAPI, HTTPException

app = FastAPI()

@app.get("/")
def handle():
    return {
        name : logging.getLevelName(logger.level))
        for name, logger in 
        logging.root.manager.loggerDict.items()
    }

Overwriting logging_files/app.py


So let's check what does we have.

In [108]:
response = get("http://localhost:8000")
response.content

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a652f961cc0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [97]:
!docker logs test_container

Error response from daemon: No such container: test_container


In [58]:
import logging
logging.root.manager.loggerDict["requests"]

True

In [99]:
import logging

# Create a logger
logger = logging.getLogger('my_logger')

# Set the level of the parent logger
logger.setLevel(logging.ERROR)

logging.getLevelName(logger.level)

'ERROR'

In [104]:
logging.getLevelName(logging.root.manager.loggerDict["requests"].level)

'NOTSET'